## URL Encoding

20.02.14  
  
Url을 Encoding한 후, url query에 city_id를 추가하는 작업

### Library 

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from urllib import parse

### Function

Encoder

In [2]:
def change_url(before):
    # Url parsing
    url = parse.urlparse(before)
    
    # query를 파싱해서 딕셔너리로 반환
    query = parse.parse_qs(url.query)
    
    # Encoding
    result = parse.urlencode(query, doseq=True)
    return result

Text changer

In [3]:
def change_text(before, after, data):
    data = data.apply(lambda x: str(x).replace(before, after))
    return data

--- 

### Mbl Url

* Load Data

In [4]:
# mbl_url 데이터 불러오기
mbl_url = pd.read_csv('Mybucketlist_url.csv', encoding = 'utf-8')

# 데이터 예시
list(mbl_url['before'].sample())

['https://findmybucketlist.com/search/newresult?keyword=자르간스&category=stay']

 * http://인 것과 https://인 것 구분

In [5]:
mbl_url['s'] = mbl_url['before'].apply(lambda x: 1 if 'https' in x else 0)

* Encoding

In [6]:
# UTF-8 Encoding
mbl_url['after'] = mbl_url['before'].apply(lambda x : change_url(x))

# Encoding 결과
list(mbl_url['after'].sample())

['keyword=%EC%9E%90%EC%B9%B4%EB%A5%B4%ED%83%80&category=stay']

* 연결자를 +가 아닌 %20으로 문자 변경

In [7]:
mbl_url['after'] = change_text('+', '%20', mbl_url['after'])

* http:s//일 경우와 http://일 경우를 구분지어 url 생성

In [8]:
# url 생성
for i in range(len(mbl_url)):   
    if mbl_url['s'][i] == 1:
        # s가 있었던 경우에는 https~
        mbl_url['after'][i] = 'https://findmybucketlist.com/search/newresult?' + mbl_url['after'][i]
    else:
        # s가 없던 경우에는 http~
        mbl_url['after'][i] = 'http://findmybucketlist.com/search/newresult?' + mbl_url['after'][i]

# url 생성 결과   
list(mbl_url['after'].sample())

['https://findmybucketlist.com/search/newresult?keyword=%EC%98%A4%EB%8B%A4%EC%99%80%EB%9D%BC&category=stay']

* cityid를 붙이기 위한 'name' 뽑아내기  

In [9]:
mbl_url['name'] = mbl_url['before'].apply(lambda x: parse.parse_qsl(parse.urlparse(x).query)[0][1])

    1) parse.urlparse(x).query = url을 parsing한 후 query만 뽑아낸다.  
    2) parse.parse_qsl(...)[0][1] = 뽑아낸 query를 리스트로 만들고 첫번째의 value를 뽑아낸다.  

In [10]:
list(mbl_url['before'].sample())

['https://findmybucketlist.com/search/newresult?keyword=디종&category=stay']

↓ 뽑아내기

In [11]:
mbl_url['name'].sample(3)

3481    화이트호스
3759       교토
7117       멜크
Name: name, dtype: object

---  

* City_Id Data

In [12]:
# city_id 데이터 불러오기 
city = pd.read_excel('city_id.xlsx')

# id가 존재하는 것만 남기기
city = city[city['id'].notnull()]

# city_id를 기준으로 중복 제거
city = city.drop_duplicates(['id'], keep = 'first').reset_index(drop = True)

# city_id 예시
city.sample(3)

,name,id
1245,호닝스버그,-263884.0
1382,아사히가와,-225793.0
1181,프리몬트,20013069.0


* 만들어진 City Id와 합치기

In [13]:
# mbl_url + city
mbl_url = pd.merge(mbl_url, city, how = 'left', on = 'name')

# 합쳐진 예시
mbl_url[['before', 'after', 'id', 'name']].sample(3)

,before,after,id,name
156,https://findmybucketlist.com/search/newresult?...,https://findmybucketlist.com/search/newresult?...,NaN,벨리
7507,https://findmybucketlist.com/search/newresult?...,https://findmybucketlist.com/search/newresult?...,-1038078.0,도네츠크
6033,https://findmybucketlist.com/search/newresult?...,https://findmybucketlist.com/search/newresult?...,-782815.0,디바


* id가 float으로 로드되었는데, 없는거 제외하고 int로 바꿔주기

In [14]:
# mbl_url['id']가 현재 float임
mbl_url['id'].dtypes

dtype('float64')

In [15]:
# nan이면 비워주고, 아니면 int로
mbl_url['id'] = mbl_url['id'].apply(lambda x: '' if str(x)=='nan' else str(int(x)))

* 최종 url 생성하기  
    1) city_id가 있는 것은 **&city_id=id값**을 붙여준다.  
    2) city_id가 없는 것은 그냥 놔둔다.  

In [16]:
mbl_url['result'] = 'not_yet'

for i in range(len(mbl_url)):
    if mbl_url['id'][i] == '':
        # id가 없다면 그냥 [after url]값 가져오기
        mbl_url['result'][i] = mbl_url['after'][i]
    else:
        # id가 있으면 [after url] + '&city_id' + [city_id]
        mbl_url['result'][i] = mbl_url['after'][i] + '&city_id=' + mbl_url['id'][i]

# result 결과
list(mbl_url['result'].sample())

['https://findmybucketlist.com/search/newresult?keyword=%EB%B9%84%EC%97%94%ED%8B%B0%EC%95%88&category=stay&city_id=-2669740']

* 내보내기

In [17]:
mbl_url.to_excel('Url_change_result.xlsx', index = False)